# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 21, 7, 0

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = True

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [15, 20, 25, 30, 35]},
              'STOCH': {'fastk_period': [5, 7, 9, 11], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [3, 5, 7, 9], 'low_bound': [10, 15, 20, 25]}}
# optim_dict = {'RSI': {'timeperiod': [16], 'low_bound': [35]},
#                   'STOCH': {'fastk_period': [5], 'slowk_period': [4],
#                             'slowd_period': [5], 'low_bound': [10]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 3840


  0%|                                                                             | 0/3840 [00:00<?, ?it/s]


Load the datasets...
Binance
BinanceFutures


100%|████████████████████████████████████████████████████████████████| 3840/3840 [2:00:19<00:00,  1.88s/it]


,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
915,STOCH_RSI,12,30,9,3,3,25,83.98,81.55,80.10,76.21,74.76,77.67,77.67,76.70,75.24,71.84,64.56,65.05,74.76,77.67,78.64,78.64,79.13,81.07,80.58,81.07,83.50,84.47,81.07,80.10,0.24,0.31,0.31,0.28,0.39,0.49,0.42,0.48,0.52,0.44,0.36,0.39,0.48,0.57,0.69,0.71,0.78,0.81,0.88,0.92,0.98,0.97,1.04,1.14,206,77.751250,1596.757500
3731,STOCH_RSI,16,35,9,3,3,25,85.47,79.39,78.38,74.32,72.64,76.35,77.03,76.01,73.65,69.26,60.47,61.49,71.28,73.99,76.01,75.68,75.68,79.05,78.38,78.38,79.05,80.07,78.38,78.72,0.24,0.32,0.29,0.28,0.32,0.46,0.42,0.48,0.48,0.40,0.26,0.32,0.47,0.56,0.67,0.67,0.79,0.85,0.85,0.91,0.93,0.94,0.97,1.11,296,75.380417,1592.603333
1171,STOCH_RSI,12,35,9,3,3,25,83.60,76.18,76.63,72.13,71.91,73.48,74.16,73.26,69.89,67.42,62.92,63.82,71.46,73.48,74.83,74.16,74.61,76.85,75.51,75.51,75.73,75.73,76.18,76.40,0.20,0.26,0.27,0.25,0.29,0.38,0.35,0.39,0.36,0.32,0.27,0.32,0.42,0.50,0.61,0.62,0.64,0.72,0.75,0.81,0.77,0.81,0.88,0.95,445,73.577083,1591.802083
2451,STOCH_RSI,14,35,9,3,3,25,84.18,77.68,76.55,72.03,71.47,74.29,75.42,74.86,71.19,67.80,62.43,62.99,71.75,74.86,76.55,75.71,75.14,77.40,77.12,76.84,76.55,76.84,77.12,77.12,0.22,0.30,0.29,0.26,0.30,0.41,0.40,0.44,0.42,0.35,0.27,0.33,0.45,0.52,0.63,0.65,0.73,0.77,0.82,0.83,0.83,0.84,0.93,1.00,354,74.328750,1532.377500
1107,STOCH_RSI,12,35,7,3,3,25,82.65,75.11,75.11,71.69,72.15,74.20,74.43,73.97,71.23,67.12,63.47,64.61,73.06,73.52,74.89,73.97,75.11,76.26,74.43,74.89,74.89,75.34,73.97,75.11,0.20,0.25,0.26,0.23,0.26,0.35,0.31,0.37,0.34,0.31,0.25,0.34,0.41,0.49,0.57,0.61,0.61,0.69,0.72,0.73,0.72,0.75,0.81,0.88,438,73.382500,1481.535000
1187,STOCH_RSI,12,35,9,4,3,25,82.40,76.27,75.20,74.93,73.33,73.33,76.00,72.27,68.80,66.13,63.47,68.27,70.67,72.80,73.87,74.93,76.53,74.13,74.93,75.47,76.27,76.80,76.00,77.07,0.20,0.24,0.27,0.27,0.30,0.30,0.35,0.37,0.32,0.30,0.35,0.39,0.45,0.52,0.60,0.64,0.62,0.72,0.69,0.78,0.77,0.82,0.83,0.92,375,73.744583,1404.218750
979,STOCH_RSI,12,30,11,3,3,25,84.36,81.04,80.57,72.51,69.19,75.36,76.78,76.30,74.88,70.14,64.93,63.98,72.99,75.83,77.25,78.67,80.57,80.57,80.57,80.09,81.99,82.46,78.20,79.15,0.23,0.30,0.32,0.25,0.30,0.47,0.42,0.50,0.48,0.41,0.36,0.40,0.48,0.52,0.70,0.69,0.78,0.79,0.87,0.88,0.90,0.95,0.93,1.02,211,76.599167,1392.424167
851,STOCH_RSI,12,30,7,3,3,25,84.73,80.30,79.31,73.40,75.37,77.83,77.83,77.34,75.86,71.92,64.53,64.53,73.89,76.35,78.33,75.86,77.83,78.82,78.33,78.82,80.79,81.28,76.35,78.33,0.22,0.31,0.29,0.24,0.30,0.43,0.39,0.47,0.48,0.39,0.36,0.42,0.44,0.56,0.63,0.63,0.76,0.73,0.85,0.91,0.91,0.86,0.90,0.97,203,76.580417,1335.824583
1219,STOCH_RSI,12,35,11,2,3,25,81.12,72.89,72.49,70.48,69.08,71.29,71.49,70.88,70.88,71.29,68.27,66.47,68.67,72.09,73.90,74.50,73.49,75.50,74.50,74.50,74.50,74.50,74.50,75.30,0.20,0.25,0.27,0.28,0.29,0.33,0.36,0.39,0.41,0.38,0.37,0.36,0.40,0.48,0.57,0.60,0.63,0.68,0.76,0.78,0.80,0.84,0.87,0.88,498,72.607500,1298.535000
1123,STOCH_RSI,1

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [12], 'low_bound': [30]},
                  'STOCH': {'fastk_period': [9], 'slowk_period': [3],
                            'slowd_period': [3], 'low_bound': [25]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)